In [ ]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from PIL import Image
import numpy as np
from tqdm import tqdm

In [ ]:
# 구축된 .npy파일을 Pytorch DataLoader을 사용할 수 있도록 CUSTOM DATASET을 만듬.
import numpy as np

class CUB200(data.Dataset):
    def __init__(self, Train, transform = None):
        super(CUB200, self).__init__()
        """
        Train : bool = True
        """
        if Train == True:
            load_data = np.load('/content/drive/MyDrive/Colab Notebooks/AI/(new)[2021 Spring Class]Ajou AI by JBR/train_image.npy')
            load_label = np.load('/content/drive/MyDrive/Colab Notebooks/AI/(new)[2021 Spring Class]Ajou AI by JBR/train_label.npy')
            self.image = load_data[::]
            self.label = load_label[::]
            self.transform = transform
            
        else:
            load_data = np.load('/content/drive/MyDrive/Colab Notebooks/AI/(new)[2021 Spring Class]Ajou AI by JBR/train_image.npy')
            load_label = np.load('/content/drive/MyDrive/Colab Notebooks/AI/(new)[2021 Spring Class]Ajou AI by JBR/train_label.npy')
            self.image = load_data[::]
            self.label = load_label[::]
            self.transform = transform

    def __getitem__(self, index):
        img, target = self.image[index], self.label[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.image)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# train_data에만 data augmentaion을 적용함.
# transform_train = transforms.Compose([
#         transforms.RandomCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# transform_test = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# # # train_data에만 data augmentaion을 적용함.
# transform_train = transforms.Compose([
#         transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
#         # transforms.RandomRotation(degrees=15),
#         transforms.RandomHorizontalFlip(),
#         transforms.CenterCrop(size=224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# transform_test = transforms.Compose([
#         transforms.Resize(size=256),
#         transforms.CenterCrop(size=224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

transform_train = transforms.Compose([
    transforms.Resize((600, 600), Image.BILINEAR),
    transforms.CenterCrop((448, 448)),
    transforms.RandomHorizontalFlip(),  # only if train
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.Resize((600, 600), Image.BILINEAR),
    transforms.CenterCrop((448, 448)),
    # transforms.RandomHorizontalFlip(),  # only if train
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [ ]:
# CUSTOM DATASET을 이용하여 train_loader, test_loader을 구축.

batch_size = 7

train_loader = torch.utils.data.DataLoader(
    dataset = CUB200(True, transform = transform_train),
    batch_size = batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = CUB200(False, transform = transform_test),
    batch_size = batch_size,
    shuffle = False
)

In [ ]:
def training_model(model, criterion, optimizer, scheduler, num_epochs = 25):
    model.train()
    for epoch in range(num_epochs):
        # scheduler.step()

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # outputs = model(inputs)
            # loss = criterion(outputs, labels)

            top_n_coordinates, concat_out, raw_logits, concat_logits, part_logits, top_n_index, top_n_prob = model(inputs)
            loss = criterion(concat_logits, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 20 == 0:
                print('[%d, %5d] loss: %.7f' %
                    (epoch + 1, (i + 1), running_loss / 20))
                running_loss = 0.0
        
        train_correct = 0
        train_total = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.squeeze()
            inputs, labels = inputs.to(device), labels.to(device)
            # outputs = model(inputs)
            top_n_coordinates, concat_out, raw_logits, concat_logits, part_logits, top_n_index, top_n_prob = model(inputs)
            

            # _, predicted = torch.max(outputs.data, 1)
            _, predicted = torch.max(concat_logits, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        print('[%d epoch] Accuracy of the network on the train images: %d %%' %
              (epoch + 1, 100 * train_correct / train_total))
        
    print("End Training do it eval_accuracy")
    return model

In [ ]:
def eval_accuracy(model):
    model.eval()
    ans = []
    class_correct = list(0. for i in range(50))
    class_total = list(0. for i in range(50))

    correct = 0
    total = 0

    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images = images.squeeze()
            images, labels = images.to(device), labels.to(device)
            top_n_coordinates, concat_out, raw_logits, concat_logits, part_logits, top_n_index, top_n_prob = model(images)
            ans.append(top_n_prob[0])
            # outputs = model(images)
            _, predicted = torch.max(concat_logits, 1)
            c = (predicted == labels).squeeze()
                    
            for i in range(labels.shape[0]):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print('Accuracy of the network on test images: %d %%' % (
        100 * correct / total), correct, total)            
                
    return ans

In [ ]:
# resnet18 = models.resnet18(pretrained=True)
# resnet50 = models.resnet50(pretrained=True)
# resnet101 = models.resnet101(pretrained=True)

# alexnet = models.alexnet(pretrained=True)
# squeezenet = models.squeezenet1_0(pretrained=True)
# vgg16 = models.vgg16(pretrained=True)
# densenet = models.densenet161(pretrained=True)  # 83
# # inception = models.inception_v3(pretrained=True, transform_input=True) # kernel error
# googlenet = models.googlenet(pretrained=True)
# shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
# mobilenet_v2 = models.mobilenet_v2(pretrained=True)
# mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)
# mobilenet_v3_small = models.mobilenet_v3_small(pretrained=True)

# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)

# mnasnet = models.mnasnet1_0(pretrained=True)

ntsnet = torch.hub.load('nicolalandro/ntsnet-cub200', 'ntsnet', pretrained=True,
                       **{'topN': 6, 'device':device, 'num_classes': 200})

Downloading: "https://github.com/nicolalandro/ntsnet-cub200/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


Downloading: "https://github.com/nicolalandro/ntsnet_cub200/releases/download/0.2/nts_net_cub200.pt" to /root/.cache/torch/hub/checkpoints/nts_net_cub200.pt


In [ ]:
# model_ft = models.mnasnet1_0(pretrained=True)
model_ft = ntsnet
for param in model_ft.parameters():
    param.requires_grad_ = False

# print(model_ft)

In [ ]:
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 50)
# print(model_ft)

# num_ftrs = model_ft.classifier.in_features
# model_ft.classifier = nn.Linear(num_ftrs, 50)

In [ ]:
# model_ft.classifier = nn.Sequential(
#     nn.Linear(960,1280,bias=True),
#     nn.Hardswish(),
#     nn.Dropout(0.2, inplace=True),
#     nn.Linear(1280,50,bias=True)
# )

In [ ]:
model_ft.concat_net = nn.Linear(10240,50,bias=True)
# model_ft.concat_net = nn.Sequential(
#     nn.Linear(10240,1024,bias=True),
#     nn.ReLU(),
#     nn.Dropout(0.2, inplace=True),
#     nn.Linear(1024,50)
# )
# model_ft.pretrained_model.layer4.fc = nn.Linear(2048, 50)
# model_ft.concat_net = nn.Linear

In [ ]:
num_epochs = 10
model_ft.to(device)
criterion = nn.CrossEntropyLoss()
# criterion = nn.NLLLoss()

optimizer = optim.SGD(model_ft.concat_net.parameters(), lr = 0.001, momentum = 0.9, weight_decay=0.001)
# optimizer = optim.SGD(model_ft.parameters(), lr = 0.001, momentum = 0.9)
# optimizer = optim.Adam(model_ft.parameters())
# lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma = 0.5)
# lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,num_epochs)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60,100])

In [ ]:
model_ft = training_model(model_ft, criterion, optimizer, lr_scheduler, num_epochs)

[1,     1] loss: 0.2035376
[1,    21] loss: 4.0197356
[1,    41] loss: 3.0080081
[1,    61] loss: 1.8358436
[1,    81] loss: 1.2340704
[1,   101] loss: 0.9305139
[1,   121] loss: 0.6880503
[1 epoch] Accuracy of the network on the train images: 89 %
[2,     1] loss: 0.0293673
[2,    21] loss: 0.4718341
[2,    41] loss: 0.5169773
[2,    61] loss: 0.4956879
[2,    81] loss: 0.4107800
[2,   101] loss: 0.4099449
[2,   121] loss: 0.4375754
[2 epoch] Accuracy of the network on the train images: 91 %
[3,     1] loss: 0.0087130
[3,    21] loss: 0.2572907
[3,    41] loss: 0.2885002
[3,    61] loss: 0.3412958
[3,    81] loss: 0.4163689
[3,   101] loss: 0.3451048
[3,   121] loss: 0.3906734
[3 epoch] Accuracy of the network on the train images: 93 %
[4,     1] loss: 0.0042972
[4,    21] loss: 0.2221440
[4,    41] loss: 0.2450003
[4,    61] loss: 0.2666346
[4,    81] loss: 0.4007307
[4,   101] loss: 0.2312811
[4,   121] loss: 0.2072119
[4 epoch] Accuracy of the network on the train images: 95 %
[5, 

In [ ]:
eval_accuracy(model_ft)

Accuracy of the network on test images: 98 % 6190 6259


[tensor([2.5451, 2.1938, 1.6880, 1.3201, 0.9967, 0.7489], device='cuda:0'),
 tensor([2.4158, 2.3358, 1.6952, 1.1198, 0.8119, 0.6576], device='cuda:0'),
 tensor([1.9590, 1.3775, 1.1779, 0.7323, 0.6865, 0.2488], device='cuda:0'),
 tensor([ 1.8494,  1.4327,  0.7532,  0.4029,  0.3135, -0.0589], device='cuda:0'),
 tensor([2.4516, 2.3879, 1.8233, 1.5553, 1.5236, 1.2722], device='cuda:0'),
 tensor([2.3252, 2.2675, 1.9979, 1.8079, 1.2285, 0.7074], device='cuda:0'),
 tensor([2.2082, 1.1689, 1.0671, 1.0566, 0.6803, 0.4553], device='cuda:0'),
 tensor([2.1304, 1.5156, 1.4328, 1.2702, 1.1504, 0.6794], device='cuda:0'),
 tensor([2.3258, 1.9705, 1.6698, 1.1581, 0.8984, 0.7633], device='cuda:0'),
 tensor([ 1.5649,  1.1925,  1.1345,  0.6485, -0.5386, -1.1106], device='cuda:0'),
 tensor([2.0239, 1.8891, 1.3126, 0.8655, 0.6511, 0.2760], device='cuda:0'),
 tensor([2.3634, 2.2209, 1.6025, 1.3349, 1.2218, 1.1925], device='cuda:0'),
 tensor([ 1.9296,  1.7331,  1.3509,  1.1324,  0.7850, -0.4746], device='cuda

In [ ]:
class TestSet(data.Dataset):
    def __init__(self, transform = None):
        super(TestSet, self).__init__()
        load_test = np.load('/content/drive/MyDrive/Colab Notebooks/AI/(new)[2021 Spring Class]Ajou AI by JBR/test_image.npy')
        self.image = load_test[::]
        self.transform = transform
            

    def __getitem__(self, index):
        img = self.image[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img

    def __len__(self):
        return len(self.image)

# ac_transform_test = transforms.Compose([
#         transforms.Resize(size=256),
#         transforms.CenterCrop(size=224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

ac_transform_test = transforms.Compose([
    transforms.Resize((600, 600), Image.BILINEAR),
    transforms.CenterCrop((448, 448)),
    # transforms.RandomHorizontalFlip(),  # only if train
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

ac_test_loader = torch.utils.data.DataLoader(
    dataset = TestSet(transform = ac_transform_test),
    batch_size = batch_size,
    shuffle = False
)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [ ]:
model_ft.eval()
ans = []
per = []
with torch.no_grad():
    for i, data in enumerate(ac_test_loader):
        data = data.squeeze()
        data = data.to(device)
        top_n_coordinates, concat_out, raw_logits, concat_logits, part_logits, top_n_index, top_n_prob = model_ft(data)
        _, predicted = torch.max(concat_logits, 1)
        for p in predicted:
            ans.append(p.item())
        # print(predicted)
        for x in concat_logits:
            per.append(x)

print(ans)

[16, 12, 23, 30, 47, 32, 38, 24, 16, 30, 28, 47, 41, 20, 35, 26, 15, 8, 37, 7, 4, 22, 22, 32, 38, 1, 41, 36, 30, 34, 42, 37, 45, 35, 44, 35, 5, 7, 40, 49, 32, 37, 32, 7, 8, 43, 22, 11, 39, 38, 12, 25, 5, 6, 32, 48, 43, 31, 8, 45, 0, 5, 45, 12, 14, 26, 40, 49, 3, 9, 48, 11, 9, 35, 12, 20, 40, 36, 42, 35, 27, 41, 36, 26, 22, 6, 4, 48, 8, 48, 33, 44, 19, 14, 34, 9, 14, 21, 4, 9, 19, 35, 21, 23, 45, 44, 43, 35, 18, 49, 10, 47, 1, 48, 25, 11, 42, 31, 43, 35, 3, 1, 21, 29, 14, 10, 38, 18, 26, 0, 32, 32, 44, 1, 9, 33, 44, 37, 16, 2, 40, 38, 3, 17, 3, 24, 26, 12, 34, 41, 11, 6, 39, 44, 38, 0, 22, 49, 3, 38, 1, 13, 41, 1, 21, 41, 14, 10, 4, 33, 49, 44, 41, 19, 7, 27, 46, 31, 11, 13, 46, 20, 27, 42, 24, 25, 17, 42, 6, 42, 36, 27, 6, 12, 48, 25, 29, 5, 6, 26, 2, 13, 16, 29, 2, 18, 9, 14, 6, 4, 44, 38, 21, 26, 24, 21, 29, 46, 2, 27, 31, 22, 31, 12, 31, 2, 18, 18, 13, 24, 35, 35, 36, 37, 4, 16, 25, 20, 20, 8, 49, 24, 40, 13, 19, 38, 49, 6, 6, 28, 31, 34, 11, 48, 1, 44, 10, 33, 33, 10, 29, 9, 27, 47

In [ ]:
result_txt = list_data = [str(int(a)).strip('\n\r') for a in ans]
print(result_txt)
print(len(result_txt))

['16', '12', '23', '30', '47', '32', '38', '24', '16', '30', '28', '47', '41', '20', '35', '26', '15', '8', '37', '7', '4', '22', '22', '32', '38', '1', '41', '36', '30', '34', '42', '37', '45', '35', '44', '35', '5', '7', '40', '49', '32', '37', '32', '7', '8', '43', '22', '11', '39', '38', '12', '25', '5', '6', '32', '48', '43', '31', '8', '45', '0', '5', '45', '12', '14', '26', '40', '49', '3', '9', '48', '11', '9', '35', '12', '20', '40', '36', '42', '35', '27', '41', '36', '26', '22', '6', '4', '48', '8', '48', '33', '44', '19', '14', '34', '9', '14', '21', '4', '9', '19', '35', '21', '23', '45', '44', '43', '35', '18', '49', '10', '47', '1', '48', '25', '11', '42', '31', '43', '35', '3', '1', '21', '29', '14', '10', '38', '18', '26', '0', '32', '32', '44', '1', '9', '33', '44', '37', '16', '2', '40', '38', '3', '17', '3', '24', '26', '12', '34', '41', '11', '6', '39', '44', '38', '0', '22', '49', '3', '38', '1', '13', '41', '1', '21', '41', '14', '10', '4', '33', '49', '44', '41'

In [ ]:
pip install pycryptodomex --no-binary :all:

     |████████████████████████████████| 3.8MB 13.9MB/s 
Skipping wheel build for pycryptodomex, due to binaries being disabled for it.
    Running setup.py install for pycryptodomex ... done


In [ ]:
import json
from base64 import b64encode
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad

def read_txt(fileName):
    with open(fileName, 'rt') as f:
        list_data = [a.strip('\n\r') for a in f.readlines()]
    return list_data

def write_json(fileName, data):
    with open(fileName, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_key(key_path):
    with open(key_path, "rb") as f:
        key = f.read()
    return key

def encrypt_data(key_path, ans_list, encrypt_store_path='ans.json'):
    key = load_key(key_path)
    data = " ".join([str(i) for i in ans_list])
    encode_data = data.encode()
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(encode_data, AES.block_size))
    iv = b64encode(cipher.iv).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    write_json(encrypt_store_path, {'iv':iv, 'ciphertext':ct})

if __name__=="__main__":
    # 1.이메일을 통해서 전달 받은 키 파일의 경로 입력
    key_path = "/content/drive/MyDrive/Colab Notebooks/AI/team12.pem"
    # 2. 예측한 결과를 텍스트 파일로 저장했을 경우 리스트로 다시 불러오기
    # 본인이 원하는 방식으로 리스트 형태로 예측 값을 불러오기만 하면 됨(순서를 지킬것)
    #raw_ans_path = "ans.txt"
    #ans = read_txt(raw_ans_path)
    ans = result_txt
    # 3. 암호화된 파일을 저장할 위치
    encrypt_ans_path = "/content/drive/MyDrive/Colab Notebooks/AI/resultfile/final6.json"
    # 4. 암호화!(pycrytodome 설치)
    encrypt_data(key_path, ans, encrypt_ans_path)

In [ ]:
len(per)

500

In [ ]:
print(per)

[tensor([ 2.0834e+00,  9.0637e-01, -3.1169e+00,  2.7966e-01, -6.5821e-01,
        -2.0875e+00, -2.5759e+00, -2.7726e+00,  3.0713e+00, -4.3710e+00,
        -2.0761e+00, -1.2432e-01, -3.0814e+00,  8.0326e-01, -1.3235e+00,
         9.0560e+00,  1.3928e+01,  9.8974e+00,  6.6275e+00,  1.9391e+00,
         7.6984e-03, -3.3286e-01, -2.0566e-01, -4.9254e-01, -1.0774e+00,
        -2.3136e+00, -3.3358e-01, -9.1404e-01,  8.5092e-02, -3.8622e+00,
        -1.3088e+00, -1.5361e+00, -3.2932e+00, -2.1793e+00, -2.9157e+00,
         3.3680e-01, -3.5524e+00,  1.8904e-01,  1.1370e+00, -5.9156e-01,
        -4.5954e-01, -2.0210e+00,  3.5849e+00, -1.9811e+00, -4.2205e+00,
         3.4437e+00, -8.5707e-01,  9.2660e-01,  3.0401e+00, -1.3925e+00],
       device='cuda:0'), tensor([ 3.9150e+00, -5.5511e-01,  1.3781e-01,  3.5341e+00, -2.6080e+00,
         1.8417e+00,  2.9726e-01, -3.8426e+00, -1.2107e+00, -7.4380e-01,
        -2.3344e+00,  2.6257e-01,  1.5479e+01, -3.6392e+00,  1.0646e+00,
         4.9988e+00, -3.

In [ ]:
preds = np.empty((500, 50))
for i in range(len(per)):
  pred = per[i].to('cpu').numpy()
  preds[i] = pred

print(preds)
np.savetxt("/content/drive/MyDrive/Colab Notebooks/AI/resultfile/preds.txt", preds, "%f")

np_ans = np.zeros(500)
for i in range(500):
  np_ans[i] = ans[i]
np.savetxt("/content/drive/MyDrive/Colab Notebooks/AI/resultfile/ans.txt", np_ans, "%d")


[[ 2.08340287  0.90637279 -3.11688542 ...  0.92659748  3.04006624
  -1.39251792]
 [ 3.91499233 -0.55510885  0.13780639 ... -1.26427591  0.06109087
   3.04099917]
 [-2.94218779  0.51030183  1.22650921 ...  0.68620175 -1.91089296
  -0.59682184]
 ...
 [ 2.17060757  6.61366606 -7.02746964 ... -1.11347044  4.52310705
   6.88569164]
 [-0.58071774 -0.35474905  2.48531437 ...  4.30119562 -1.81567872
  -3.54844451]
 [-1.10730612  1.66106081 -1.26290739 ... -1.15473557  1.35107851
   3.24817991]]
